In [ ]:
from __future__ import division, print_function

import skimage
import numpy as np
from matplotlib import pyplot as plt
from skimage import img_as_float, img_as_ubyte
from skimage import io

from scipy import fftpack

%matplotlib inline

In [ ]:
img = io.imread('bossier.jpg')
img = img_as_float(img)

In [ ]:
def pad_image(img):
    ro = img.shape[0]
    co = img.shape[1]
    r = ro + ro % 8
    c = co + co % 8
    if len(img.shape) == 3:
        ch = img.shape[2]
        img2 = np.zeros(r*c*ch).reshape(r,c,ch)
        img2[:ro,:co,:] = img[:ro,:co,:]
    else: # single channel
        img2 = np.zeros(r*c).reshape(r,c)
        img2[:ro,:co] = img[:ro,:co]
    return img2

In [ ]:
img = pad_image(img)

print(img.shape, type(img))

fig, axes = plt.subplots(ncols=1, figsize=(10, 10))
axes.imshow(img)
axes.set_xticks([])
axes.set_yticks([]);


In [ ]:
R = img.copy()
G = img.copy()
B = img.copy()
R[:,:,(1,2)] = 0
G[:,:,(0,2)] = 0
B[:,:,(0,1)] = 0

fig, axes = plt.subplots(ncols=3, figsize=(10, 10))
axes[0].imshow(R)
axes[1].imshow(G)
axes[2].imshow(B);

In [ ]:
def get_2D_dct(img):
    """ Get 2D Cosine Transform of Image
    """
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

def get_2D_idct(coefficients):
    """ Get 2D Inverse Cosine Transform of Image
    """
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

def get_8x8_block(img, r_start, c_start, channel):
    """ Get an 8x8 block from the image
    """
    return img[r_start:r_start+8, c_start:c_start+8, channel]

def put_8x8_block(img, block, r_start, c_start, channel):
    img_new = img
    img_new[r_start:r_start+8, c_start:c_start+8, channel] = block
    return img_new

def compress(img, n_coeff):
    img_reconstructed = np.zeros_like(img)

    for start_row in np.arange(0,img.shape[0],8):
        for start_col in np.arange(0,img.shape[1],8):
            for channel in range(0,3):
                block = get_8x8_block(img, start_row, start_col, channel)
                block_dct = get_2D_dct(block)
                # block_dct_quantized = quantize(block_dct, factor)
                block_dct_quantized = get_n_max_vals(block_dct, int(n_coeff))
                block_reconstructed = get_2D_idct(block_dct_quantized)
                img_reconstructed = put_8x8_block(img_reconstructed, block_reconstructed, start_row, start_col, channel)
    return np.clip(img_reconstructed,0,1)

def get_n_max_vals(blk, n):
    # Convert it into a 1D array and take absolute value
    a_1d = np.abs(blk.flatten())
    # Find the indices in the 1D array
    idx_1d = a_1d.argsort()[-n:]
    # Get min of the maxes
    m = a_1d[idx_1d[0]]
    
    b = np.zeros_like(blk)
    b[blk >= m] = blk[blk >= m] # Large positive values
    b[blk <= -m] = blk[blk <= -m] # Large negative values
    return b


## Converting to YCbCr

https://en.wikipedia.org/wiki/YCbCr

JFIF usage of JPEG allows Y′CbCr where Y′, CB and CR have the full 8-bit range of 0-255:[4]

$$
    \begin{align} Y' &=& 0 &+ (0.299 & \cdot R'_D) &+ (0.587 & \cdot G'_D) &+ (0.114 & \cdot B'_D)\\ C_B &=& 128 & - (0.168736 & \cdot R'_D) &- (0.331264 & \cdot G'_D) &+ (0.5 & \cdot B'_D)\\ C_R &=& 128 &+ (0.5 & \cdot R'_D) &- (0.418688 & \cdot G'_D) &- (0.081312 & \cdot B'_D) \end{align}
$$

And back:

$$
    \begin{align} R &=& Y &&& + 1.402 & \cdot (C_R-128) \\ G &=& Y & - 0.34414 & \cdot (C_B-128)& - 0.71414 & \cdot (C_R-128) \\ B &=& Y & + 1.772 & \cdot (C_B-128)& \end{align} 
$$

In [ ]:
def rgb2ycbcr(img):
    image_ubyte = img_as_ubyte(img)
    img_new = np.zeros_like(image_ubyte)
    CM_to_ycbcr = np.array([[0, .299, .587, .114],
                            [128, -.168736, -.331264, .5],
                            [128, .5, -.418688, -.081312]])
    
    for row in range(image_ubyte.shape[0]):
        for col in range(image_ubyte.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = image_ubyte[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_ycbcr, pix_with_bias)
    return img_as_float(img_new)

Factor out bias terms in the reverse direction:

$$
\begin{align}
R &= 1 \times Y + 0 \times (C_B - 128) + 1.402 \times (C_R-128) \\
R &= 1 \times Y + 0 C_B - 0 \times 128 + 1.402 C_R - 1.402 \times 128 \\
R &= - 1.402 \times 128 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
R &= - 179.456 + 1 \times Y + 0 \times C_B + 1.402 \times C_R \\
\\
G &= 1 \times Y - 0.34414 \times (C_B - 128) - 0.71414 \times (C_R-128) \\
G &= 1 \times Y - 0.34414 \times C_B + 0.34414 \times 128 - 0.71414 \times C_R + 0.71414 \times 128 \\
G &= (0.34414 + 0.71414) \times 128 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
G &= 135.45984 + 1 \times Y - 0.34414 \times C_B  - 0.71414 \times C_R \\
\\
B &= 1 \times Y + 1.772 \times (C_B - 128) + 0 \times (C_R-128) \\
B &= 1 \times Y + 1.772 \times C_B - 1.772 \times 128 + 0 \times C_R \\
B &= - 1.772 \times 128 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
B &= - 226.816 + 1 \times Y + 1.772 \times C_B + 0 \times C_R \\
\end{align}
$$



In [ ]:
def ycbcr2rgb(img):
    image_ubyte = img_as_ubyte(img)
    img_new = np.zeros_like(image_ubyte)
    CM_to_rgb = np.array([[-179.456, 1, 0, 1.402],
                          [135.45984, 1, -.34414, -.71414],
                          [-226.816, 1, 1.772, 0]])

    for row in range(image_ubyte.shape[0]):
        for col in range(image_ubyte.shape[1]):
            pix_with_bias = np.ones(4)
            pix_with_bias[1:4] = image_ubyte[row, col, :]
            img_new[row, col, :] = np.dot(CM_to_rgb, pix_with_bias)
            # if np.any(img_new[row, col, :] < 0) |  np.any(img_new[row, col, :] > 255):
                # print(row, col, pix_with_bias, img_new[row, col, :])
    # return img_as_float(img_new)    
    return img_as_float(np.clip(img_new, 0, 255))

In [ ]:
img_ycbcr = ycbcr2rgb(img)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15, 15))
labels = ["Y", "Cb", "Cr"]
for ch in range(0,3):
    axes[ch].imshow(img_ycbcr[:,:,ch], cmap="gray", interpolation="nearest")
    axes[ch].set_xlabel("Channel: %s" % labels[ch], fontsize=18)
    axes[ch].set_xticks([])
    axes[ch].set_yticks([])

plt.tight_layout()

In [ ]:
n_coeff = 32.0

fig, axes = plt.subplots(2, 3, figsize=(15, 15))
fig.suptitle("Image Results -- N Largest DCT Coefficients", fontsize=24)
for r in range(0,2):
    for c in range(0,3):
        if (r==0) & (c==0) & False: 
            axes[r,c].imshow(img)
        else: 
            img_compressed = compress(img, n_coeff)
            axes[r,c].imshow(img_compressed[100:500, 200:500, :], interpolation="nearest")
            # axes[r,c].set_title("%s/64 DCT Coefficients" % int(n_coeff))
            axes[r,c].set_xlabel("%s/64 Coefficients" % int(n_coeff), fontsize=18)
            axes[r,c].set_xticks([])
            axes[r,c].set_yticks([])
            fname = "b_rgb_" + str(n_coeff) + ".jpg"
            io.imsave(fname, img_as_ubyte(img_compressed))
        n_coeff = n_coeff/2
    
plt.tight_layout()
                                